In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun  5 13:39:01 2025

@author: aydina

Script to calculate priority scores for sampled 266 businesses

Updates:
- User is asked to define period
- Output file added
- Filepaths changed
- Rank-based normalization added
"""

import pandas as pd
import numpy as np
import os
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Prompt user to define the period
period = input("Please enter the period (e.g., 202506): ").strip()

# Validate period format (basic check)
if not (period.isdigit() and len(period) == 6):
    raise ValueError("Invalid period format. Please enter a 6-digit numeric value like '202506'.")

# Define file paths using the user-defined period
base_path = r'\\cr2-data1\BDOD_Redesign\Survey Workflow Tools\FSS\files_System'
dim_path_universe_file = os.path.join(base_path, period, f'universeConsolidated_266_{period}.csv')
dim_path_output_file = os.path.join(base_path, period, f'priority_scores_output_{period}.csv')

try:
    # Read universe file
    universe = pd.read_csv(dim_path_universe_file)

    # Check for required columns
    required_columns = ['ruref', 'frosic2007', 'cell_no', 'selmkr', 'Total Assets']
    missing_columns = [col for col in required_columns if col not in universe.columns]
    if missing_columns:
        raise ValueError(f"Missing required columns: {missing_columns}")

    # Calculate the size of each cell in the universe
    cell_size_universe = universe['cell_no'].value_counts().sort_index()
    cell_size_universe.name = 'universe'

    # Filter non-selected businesses
    selection = universe[universe['selmkr'] != 'N'][required_columns]

    # Calculate the size of each cell in the sample
    cell_size_sample = selection['cell_no'].value_counts().sort_index()
    cell_size_sample.name = 'sample'

    # Merge cell_size series
    cell_df = pd.concat([cell_size_sample, cell_size_universe], axis=1)

    # Avoid division by zero
    cell_df['sample'] = cell_df['sample'].replace(0, np.nan)

    # Calculate the design weight
    cell_df['weight'] = cell_df['universe'] / cell_df['sample']

    # Merge with selection
    df = pd.merge(selection, cell_df, left_on='cell_no', right_index=True)

    ####### Calculate the priority score #######
    # Baseline score
    df['base_score'] = df['weight'] * df['Total Assets']

    # Rank-Based Normalization (Percentile Ranking)
    df['group'] = df['frosic2007'].astype(str) + df['selmkr'].apply(lambda x: '_ref' if x == 'L' else '_noref')
    df['normalised_score'] = df.groupby('group')['base_score'].rank(pct=True)

    # Save the final DataFrame to CSV
    df.to_csv(dim_path_output_file, index=False)
    logging.info(f"Priority scores saved to: {dim_path_output_file}")

except FileNotFoundError:
    logging.error(f"File not found: {dim_path_universe_file}")
except Exception as e:
    logging.error(f"An error occurred: {e}")
